In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9989.000000,10000.000000,10000.000000,9890.000000
mean,7.297602,27.298900,1.034400,38.699798
std,1.291685,6.165838,0.192926,2.539957
min,0.612885,12.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.257702,50.000000,3.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5150
False    4850
Name: is_male, dtype: int64

In [6]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [7]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])

In [8]:
data['is_male'] = data['is_male'].astype(int)

In [9]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [10]:
model = xgb.XGBRegressor(
    objective='reg:squarederror'
)

In [11]:
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
y_pred = model.predict(x_test)

In [14]:
for i in range(20):
    print('Predicted weight: ', y_pred[i])
    print('Actual weight: ', y_test.iloc[i])
    print()

Predicted weight:  8.000681
Actual weight:  7.12534030784

Predicted weight:  7.1026506
Actual weight:  7.054792384

Predicted weight:  7.757792
Actual weight:  9.0609989682

Predicted weight:  6.55763
Actual weight:  6.2170357884

Predicted weight:  7.5132627
Actual weight:  6.99306295064

Predicted weight:  7.7035155
Actual weight:  8.000575487979999

Predicted weight:  5.1863112
Actual weight:  3.56267015392

Predicted weight:  8.001055
Actual weight:  7.43839671988

Predicted weight:  7.8372355
Actual weight:  9.92080179

Predicted weight:  7.6199713
Actual weight:  8.98604179912

Predicted weight:  2.2529392
Actual weight:  5.8642961692

Predicted weight:  8.081053
Actual weight:  7.9035720927

Predicted weight:  7.1319327
Actual weight:  6.5918216337999995

Predicted weight:  6.5902176
Actual weight:  7.43839671988

Predicted weight:  7.597389
Actual weight:  8.56275425608

Predicted weight:  6.659524
Actual weight:  7.4846937949

Predicted weight:  7.8372355
Actual weight:  8.93

In [15]:
model.save_model('model.bst')

In [16]:
!gcloud config list project --format "value(core.project)"

modern-baton-353217


In [17]:
# Update these to your own GCP project, model, and version names
GCP_PROJECT = 'modern-baton-353217'
MODEL_BUCKET = 'gs://modern-baton-353217'
VERSION_NAME = 'v1'
MODEL_NAME = 'baby_weight'

In [18]:
!gsutil mb $MODEL_BUCKET

Creating gs://modern-baton-353217/...


In [19]:
!gsutil cp ./model.bst $MODEL_BUCKET

Copying file://./model.bst [Content-Type=application/octet-stream]...
/ [1 files][308.0 KiB/308.0 KiB]                                                
Operation completed over 1 objects/308.0 KiB.                                    


In [20]:
!gcloud ai-platform models create $MODEL_NAME --region=us-central1

Using endpoint [https://us-central1-ml.googleapis.com/]
Created ai platform model [projects/modern-baton-353217/models/baby_weight].


In [ ]:
!gcloud ai-platform versions create $VERSION_NAME \
--model=$MODEL_NAME \
--framework='XGBOOST' \
--runtime-version=2.5 \
--origin=$MODEL_BUCKET \
--python-version=3.7 \
--project=$GCP_PROJECT \
--region=us-central1

Using endpoint [https://us-central1-ml.googleapis.com/]
Creating version (this might take a few minutes)......⠶                        

In [ ]:
%%writefile predictions.json
[0.0, 33.0, 1.0, 27.0]
[1.0, 26.0, 1.0, 40.0]

In [ ]:
prediction = !gcloud ai-platform predict --model=$MODEL_NAME --json-instances=predictions.json --version=$VERSION_NAME
print(prediction.s)